<a href="https://colab.research.google.com/github/Alex64-1149/VoxNote/blob/IA-VoxNote/VoxNote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adaptation des données

In [114]:

#adaptation de https://github.com/musikalkemist/pytorchforaudio

import torch
import torchaudio
from torch.utils.data import Dataset
import pandas as pd


class ReconnaissanceVocaleDataset(Dataset) :

  #Initialise les aurguments de la classe dataset:
  # 1.fichier annotation(fichier txt qui contient tous les noms des fichiers audio (wav))
  # 2.fichier audio ( contient tous les fichiers audio(wav))
  def __init__(self,FICHIER_ANNOTE,FICHIER_AUDIO, melSpectrogram, SAMPLE_RATE, NOMBRE_ECHANTILLONS, processeur): # __[...]__ = classe nécessaire a un dataset Pytorch
    self.fichierAnnotations=FICHIER_ANNOTE
    self.fichierAudio=FICHIER_AUDIO
    self.processeur = processeur
    self.melSpectrogram = melSpectrogram.to(processeur) #s'assurer que tout se fasse au même endroit dans l'ordinateur : https://stackoverflow.com/questions/63061779/pytorch-when-do-i-need-to-use-todevice-on-a-model-or-tensor
    self.SAMPLE_RATE_VOULU = SAMPLE_RATE
    self.NOMBRE_ECHANTILLONS = NOMBRE_ECHANTILLONS

  #retourne le nombre de fichiers dans notre dataset
  def __len__(self):
    return len(self.fichierAnnotations)

  #retourne l'audio ainsi que son fichier texte associé
  def __getitem__(self, index):
    pathAudio = self.getAudioSamplePath(index)
    texte = self.getAudioSampleText(index)
    #signal informatique et sample rate de notre audio
    signal, sr = torchaudio.load(pathAudio)

    #mettre le signal au même endroit que sa transformation
    signal = signal.to(self.processeur)

    #normaliser l'audio
    signal = self.resampleSiNecessaire(signal, sr)
    signal = self.combinerSiNecessaire(signal)
    #diminuer ou ajouter des échantillons "vides" si le nombre d'échantillons ne correspond à 22 050
    signal = self.couperSiNecessaire(signal)
    signal = self.paddingSiNecessaire(signal)

    #transformer l'audio dans le spectogram de mel
    signal = self.melSpectrogram(signal)
    return signal, texte

  #mettre tous les fichiers audios à la même fréquence d'échantillonage
  def resampleSiNecessaire(self, signal, sr):
    if sr != self.SAMPLE_RATE_VOULU :
      resampler = torchaudio.transforms.Resample(sr, self.SAMPLE_RATE_VOULU)
      signal = resampler(signal)
    return signal

  #s'assurer que l'audio ne contient qu'une entrée et sortie (que le son ne soit pas stéréo) pour le normaliser
  def combinerSiNecessaire(self, signal):
    if signal.shape[0] > 1 :
      signal = torch.mean(signal, dim=0, keepdim= True)
    return signal

  #enlever les échantillons audios superflus
  def couperSiNecessaire(self, signal) :
    #le signal est composé de 2 dimensions : [nombre de source du signal(1 dans ce cas), longueur du signal – nombre d'échantillons (on veut 22 050)]
    if signal.shape[1] > self.NOMBRE_ECHANTILLONS :
      signal = signal[:, :self.NOMBRE_ECHANTILLONS] #utilité de [:, :] : la première dimension est prise au complet et la deuxième jusqu'à l'atteinte du nombre d'échantillon (expliquer à https://youtu.be/WyJvrzVNkOc?list=PL-wATfeyAMNoirN4idjev6aRu8ISZYVWm&t=478)
    return signal

  def paddingSiNecessaire(self, signal) :
    #la longueur du signal = signal.shape[1] comme expliqué précédemment
    if signal.shape[1] < self.NOMBRE_ECHANTILLONS :
      paddingDuSignal = (0, self.NOMBRE_ECHANTILLONS - signal.shape[1]) #(0, nombre d'échantillons manquants)
      torch.nn.functional.pad(signal, paddingDuSignal) #ajoute le padding au signal (https://pytorch.org/docs/stable/generated/torch.nn.functional.pad.html)
    return signal


  # TODO: vérifier fichierAurio[index] renvoie quoi
  #retourne le chemin pour avoir le bon fichier audio à un certain index du FICHIER_ANNOTE
  def getAudioSamplePath(self, index):
    return self.fichierAudio[index]

  #retourne le fichier texte associé à l'audio d'un certain index du FICHIER_AUDIO
  def getAudioSampleText(self, index):
    return self.fichierAnnotations[index]


#FICHIER_ANNOTE = "/content/drive/MyDrive/Colab Notebooks/SiwisFrenchSpeechSynthesisDatabase/lists/all_text.list"
#FICHIER_AUDIO = "/content/drive/MyDrive/Colab Notebooks/SiwisFrenchSpeechSynthesisDatabase/lists/all_wavs.list"
#
#
##nombre d'échantillons par secondes dans notre audio
#SAMPLE_RATE = 22050
#NOMBRE_ECHANTILLONS = 22050
#
#if torch.cuda.is_available(): #détermine ce qui exécute le programme (gpu préférable pour AI audio)
#    processeur = "cuda"
#else:
#    processeur = "cpu"
#print(f"Utiliation du processeur {processeur}")
#
##le Spectogram de Mel est une échelle logarithmique utilisée pour mieux représenter les différences qu'un humain entend dans un fichier audio ce qui aide à l'analyse sonore
#melSpectrogram = torchaudio.transforms.MelSpectrogram(
#    SAMPLE_RATE,
#    n_fft=512, #longueur physique du signal optimale pour la reconnaissance vocale selon : https://librosa.org/doc/main/generated/librosa.stft.html
#    hop_length=512, #nombre d'échantillon audio adjacents analysés par la transformée de fourier : https://librosa.org/doc/main/generated/librosa.stft.html
#    n_mels=64 #nombre de séparations d'une seule fréquence optimale pour la reconnaisance vocale selon:https://stackoverflow.com/questions/62623975/why-128-mel-bands-are-used-in-mel-spectrograms
#)
#
#
#rvd = ReconnaissanceVocaleDataset(FICHIER_ANNOTE, FICHIER_AUDIO, melSpectrogram, SAMPLE_RATE, NOMBRE_ECHANTILLONS, processeur)


In [115]:
from google.colab import drive #nécessaire qu début de chaque session
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
#utilisé pour extraire le zip de google drive
#from google.colab import drive
#drive.mount('/content/drive')


#!unzip file.zip


# Réseau de neurone

In [117]:
#connaissances nécessaires au RNN(LSTM) trouvées à l'adresse suivante : https://ketanhdoshi.github.io/Audio-ASR/
#CNN initial vient de https://www.youtube.com/watch?v=SQ1iIKs190Q&list=PL-wATfeyAMNoirN4idjev6aRu8ISZYVWm&index=8

import torch
import torchaudio
from torch import nn
import numpy as np
!torch -m pip install torchsummary
from torchsummary import summary

class BiRNNetwork(nn.Module) : #RNN de type bidirectional LSTM

  def __init__(self, n_caracteres, rnn_dim, n_lstm_layers, dropout) :
    super().__init__()
    # 5 RNN blocks / flatten / linear /softmax
    """
    1. Créer le bi-lstm (une sous catégorie des RNN) : https://medium.com/@anishnama20/understanding-bidirectional-lstm-for-sequential-data-processing-b83d6283befc
    2. 'flatten' le résultat en diminuant le nombre de dimensions créées avec les blocs du RNN
    3. transformer en équation linéaire les données fournies : https://docs.kanaries.net/topics/Python/nn-linear
    4. normaliser les résultats à l'aide de softmax
    """
    self.n_caracteres = n_caracteres
    self.rnn_dim = rnn_dim
    self.n_lstm_layers = n_lstm_layers
    self.dropout = dropout
    #self.batchSize = batchSize #nombre d'échantillon avant de changer les paramètres : https://datascience.stackexchange.com/questions/36651/relationship-between-batch-size-and-the-number-of-neurons-in-the-input-layer


    """self.conv1 = nn.Sequential(
        nn.Dropout(0.1),#réduit les chances que le réseau de neurone s'adapte à une seule circonstance précise https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
        nn.Conv2d( #couche en 2 dimensions
            in_channels=inChannels, #nombre de input initial (=1 lors de l'adaptation des données)
            out_channels=outChannels, #nombre de filtre dans cette couche du réseau de neurone
            kernel_size=3, #nombre de choses analysées en même temps : https://stats.stackexchange.com/questions/296679/what-does-kernel-size-mean
            stride = 1, #déplacement du kernel : https://deepai.org/machine-learning-glossary-and-terms/stride
            padding = 2 #comme dans RSD mais pour le kernel
        ),
        nn.GELU(), #régression linéaire Gaussienne https://stackoverflow.com/questions/57532679/why-gelu-activation-function-is-used-instead-of-relu-in-bert
        nn.MaxPool2d(kernel_size=2) # reformulation des données : https://www.geeksforgeeks.org/apply-a-2d-max-pooling-in-pytorch/
    )"""

    self.layerNorm = nn.LayerNorm(rnn_dim) #réduit les valeurs des paramètres de chaque neurone du réseau pour faciliter la descente de gradient : https://www.youtube.com/watch?v=TKPowx9fb-A

    self.conv1 = self.sequence()
    self.conv2 = self.sequence()
    self.conv3 = self.sequence()
    self.conv4 = self.sequence()
    self.conv5 = self.sequence()

    self.flatten = nn.Flatten()
    self.linear = nn.Linear(n_caracteres *5 * 2, #input (out_channels final, fréquence, temps)
                           n_caracteres #nb de output
                           )
    self.softmax = nn.Softmax(dim=1)


  def sequence(self) :
    conv = nn.Sequential(
      nn.LSTM(self.rnn_dim, #nombre de input initial
              num_layers=self.n_lstm_layers, #nombre de couches passées à travers avant de retourner une valeur : https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm
              hidden_size= self.rnn_dim, #nombre de composantes des vecteurs représentant les valeurs : https://stackoverflow.com/questions/75648914/trying-to-understand-lstm-parameter-hidden-size-in-pytorch#:~:text=The%20hidden_size%20is%20a%20hyper,hyper%2Dparameter%20(%20num_layers%20).
              dropout=self.dropout, #réduit les chances que le réseau de neurone s'adapte à une seule circonstance précise https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
              bidirectional=True),
      nn.GELU(), #régression linéaire Gaussienne https://stackoverflow.com/questions/57532679/why-gelu-activation-function-is-used-instead-of-relu-in-bert
    )
    return conv

  def forward(self, input_data): #traitement des données dans le réseau de neurone
    x = self.layerNorm(input_data)
    x = self.conv1(x) #RNN de 5 de profondeurs
    x = self.layerNorm(x)
    x = self.conv2(x)
    x = self.layerNorm(x)
    x = self.conv3(x)
    x = self.layerNorm(x)
    x = self.conv4(x)
    x = self.layerNorm(x)
    x = self.conv5(x)
    #x = self.conv1(input_data)
    ##passer le x des convolutional layers vers le flatten
    #x = self.flatten(x)

    logits = self.linear(x) #logits signifie la probabilité (avant d'être normalisé) associée à certaines réponses : https://www.linkedin.com/posts/mwitiderrick_what-are-logits-in-deep-learning-logits-activity-7084819307959902209-UUGe#:~:text=Logits%20are%20the%20outputs%20of%20a%20neural%20network%20before%20the,belonging%20to%20a%20certain%20class.
    predictions = self.softmax(logits) #normaliser les logits
    return x


"""class RCNN (nn.Module):
  #Residual convolutional neural network basé sur :
    #1. https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/
    #2. https://ketanhdoshi.github.io/Audio-ASR/
  #
  def __init__(self, nbCaracteresPossibles, inChannels, outChannels) :
    super.__init__()

  def forward(self, input_data) :
    pass
"""

#algorithme pour le mapping prit sur https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/
char_map_str = """
 ' 0
 <SPACE> 1
 a 2
 b 3
 c 4
 d 5
 e 6
 f 7
 g 8
 h 9
 i 10
 j 11
 k 12
 l 13
 m 14
 n 15
 o 16
 p 17
 q 18
 r 19
 s 20
 t 21
 u 22
 v 23
 w 24
 x 25
 y 26
 z 27
 - 28
 à 29
 â 30
 ä 31
 é 32
 è 33
 ê 34
 ë 35
 î 36
 ï 37
 ô 38
 ö 39
 ù 40
 û 41
 ü 42
 ÿ 43
 ç 44
 """
#associer des caractères à des valeurs numériques
class TextTransform:
  """Maps characters to integers and vice versa"""
  def __init__(self, char_map_str):
      self.char_map_str = char_map_str
      self.char_map = {}
      self.index_map = {}
      for line in char_map_str.strip().split('\n'):
          ch, index = line.split()
          self.char_map[ch] = int(index)
          self.index_map[int(index)] = ch
      self.index_map[1] = ' '

  def text_to_int(self, text):
      """ Use a character map and convert text to an integer sequence """
      int_sequence = []
      for c in text:
          if c == ' ':
              ch = self.char_map['']
          else:
              ch = self.char_map[c]
          int_sequence.append(ch)
      return int_sequence

  def int_to_text(self, labels):
      """ Use a character map and convert integer labels to an text sequence """
      string = []
      for i in labels:
          string.append(self.index_map[i])
      return ''.join(string).replace('', ' ')


#hyper_parameters = { #paramètres qui décident comment se dérouleral l'entrainement : https://aws.amazon.com/what-is/hyperparameter-tuning/#:~:text=computationally%20intensive%20process.-,What%20are%20hyperparameters%3F,set%20before%20training%20a%20model.
#        "n_lstm_layers": 2, #nombre de couches de lstm
#        "rnn_dim": 512, #inChannels
#        "n_caracteres": 45, #dimensionOutput
#        "dropout": 0.1, ##réduit les chances que le réseau de neurone s'adapte à une seule circonstance précise https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
#        "learning_rate": 0.001, #vitesse d'apprentissage
#        "batch_size": 20, #nombre d'éléments par endoit qu'on entraine
#        "epochs": 10 #nombre de fois qu'on entraine le réseau au complet
#    }

#rnn = BiRNNetwork(hyper_parameters["n_caracteres"], hyper_parameters["rnn_dim"], hyper_parameters["n_lstm_layers"]) #dimensionOutput, inChannels, nombre de couche
#summary(rnn.cuda(), (2,1,64,44)) #(nombre de channels, frequence, temps)

/bin/bash: line 1: torch: command not found


# Entraîner et Tester

In [118]:
import torch
import torchaudio
from torch.utils.data import DataLoader

#paramètres essentiels lors de machine learning
hyper_parameters = { #paramètres qui décident comment se déroulera l'entrainement : https://aws.amazon.com/what-is/hyperparameter-tuning/#:~:text=computationally%20intensive%20process.-,What%20are%20hyperparameters%3F,set%20before%20training%20a%20model.
        "n_lstm_layers": 2, #nombre de couches de lstm
        "rnn_dim": 512, #inChannels
        "n_caracteres": 45, #dimensionOutput
        "dropout": 0.1, ##réduit les chances que le réseau de neurone s'adapte à une seule circonstance précise https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
        "learning_rate": 0.001, #vitesse d'apprentissage
        "batch_size": 20, #nombre d'éléments par endoit qu'on entraine
        "epochs": 10 #nombre de fois qu'on entraine le réseau au complet
    }

"""entrainer le RNN à l'aide du CTC Algorithm, un algorithme qui sert à déterminer où sont placer les lettres dans un fichier audio : https://ketanhdoshi.github.io/Audio-ASR/"""


def entrainerEpoque(modele, chargeurDonnees, criterion, optimizer, scheduler, epoch, processeur):
  for audio, texteAssocie in chargeurDonnees :
    #s'assurer que l'audio et le texte soit au même endroit que le modèle
    audio = audio.to(processeur)
    texteAssocie = texteAssocie.to(processeur)

    #réinitialise les paramètres pour permettre au réseau de neurone de ne pas s'encombrer de paramètres précédents inutiles : https://medium.com/@lazyprogrammerofficial/in-pytorch-why-do-we-need-to-call-optimizer-zero-grad-8e19fdc1ad2f
    optimizer.zero_grad()

    #faire une prédiction et déterminer de comment il faut modifier notre réseau de neurone, à l'aide du CTCLoss, pour améliorer la prédiction
    prediction = modele(audio)
    divergence = criterion(audio, texteAssocie)

    #update les valeurs du réseau de neurone avec une backpropagation
    divergence.backward() # modifie le poid de chaque paramètres selon la divergence calculée (est-ce qu'il faut augmenter ou diminuer la valeur de ce neurone) : https://en.wikipedia.org/wiki/Backpropagation


    #passage au prochain paramètre de l'optimizer et du scheduler
    optimizer.step() #descente de gradient : https://stackoverflow.com/questions/53975717/pytorch-connection-between-loss-backward-and-optimizer-step
    scheduler.step() #modifie le learning rate : https://discuss.pytorch.org/t/what-does-scheduler-step-do/47764

def tester(modele, chargeurDonnees, criterion, epoch, processeur, texte_transforme, caractere_vide) :

  modele.eval() # rend fixe certains paramètres qui sont changés durant l'entraînement : https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch
  divergence_test = 0
  erreurLettre = 0
  nombreLettre = 0

  with torch.no_grad() : #même chose que model.eval() sur d'autres paramètres
    for audio, texteAssocie in chargeurDonnees :
      #code similaire au train
      audio = audio.to(processeur)
      texteAssocie = texteAssocie.to(processeur)

      prediction = modele(audio)
      divergence = criterion(audio, texteAssocie)
      divergence_test += divergence

      #transformer les valeurs du spectogram en valeurs numériques et enuite en texte
      predictions_decode, valeurs_decode = GreedyDecoder(prediction, texteAssocie, texte_transforme, caractere_vide)
      nombreLettre += max(len(predictions_decode), len(valeurs_decode))
      for i in range(len(predictions_decode)) :
        if i < valeurs_decode :
          if valeurs_decode != predictions_decode :
            erreurLettre += 1

      pourcentErreur = erreurLettre/nombreLettre*100
      print(f"Le pourcentage d'erreur à l'époque {epoch} = {pourcentErreur}% pour les lettres et la divergence(loss) = {divergence_test}")

"""
fonction courante dans les speech to text qui compare le résultat attendu avec le résultat obtenu
le format de celui-ci est inspiré par : https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/
"""
def GreedyDecoder(prediction, texteAssocie, texte_transforme, caractere_vide) :
  valeursPrevuesRNN = torch.argmax(prediction) #retourne les valeurs du RNN les plus probables (maximum) #https://pytorch.org/docs/stable/generated/torch.argmax.html
  decode = []
  attendu = texteAssocie.split(" ")
  for i, valeurs in enumerate(valeursPrevuesRNN) : #valeurs correspond au caracère à chaque endroit possible
    for j, index in enumerate(valeurs) : #valeur correspond au caractère à un endroit précis
      if index != caractere_vide:
        decode.append(index)
  decode = texte_transforme.int_to_text(decode)
  return decode, valeursPrevuesRNN


FICHIER_ANNOTE = "/content/drive/MyDrive/Colab Notebooks/SiwisFrenchSpeechSynthesisDatabase/lists/all_text.list"
FICHIER_AUDIO = "/content/drive/MyDrive/Colab Notebooks/SiwisFrenchSpeechSynthesisDatabase/lists/all_wavs.list"


#nombre d'échantillons par secondes dans notre audio
SAMPLE_RATE = 22050
NOMBRE_ECHANTILLONS = 22050

if torch.cuda.is_available(): #détermine ce qui exécute le programme (gpu préférable pour AI audio)
    processeur = "cuda"
else:
    processeur = "cpu"
print(f"Utiliation du processeur {processeur}")

#le Spectogram de Mel est une échelle logarithmique utilisée pour mieux représenter les différences qu'un humain entend dans un fichier audio ce qui aide à l'analyse sonore
melSpectrogram = torchaudio.transforms.MelSpectrogram(
    SAMPLE_RATE,
    n_fft=512, #longueur physique du signal optimale pour la reconnaissance vocale selon : https://librosa.org/doc/main/generated/librosa.stft.html
    hop_length=512, #nombre d'échantillon audio adjacents analysés par la transformée de fourier : https://librosa.org/doc/main/generated/librosa.stft.html
    n_mels=64 #nombre de séparations d'une seule fréquence optimale pour la reconnaisance vocale selon:https://stackoverflow.com/questions/62623975/why-128-mel-bands-are-used-in-mel-spectrograms
)

#initialisation du Dataset
rvd = ReconnaissanceVocaleDataset(FICHIER_ANNOTE, FICHIER_AUDIO, melSpectrogram, SAMPLE_RATE, NOMBRE_ECHANTILLONS, processeur)

#https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
train_data_loader = DataLoader(rvd,
                              hyper_parameters["batch_size"],
                              shuffle=True, #https://discuss.pytorch.org/t/how-does-shuffle-in-data-loader-work/49756/7
                              )
test_data_loader = DataLoader(rvd, #préférablement pas tt le dataset(rvd)
                              hyper_parameters["batch_size"],
                              shuffle=False,
                              )

#initialisation du réseau de neurones
rnn = BiRNNetwork(hyper_parameters["n_caracteres"], hyper_parameters["rnn_dim"], hyper_parameters["n_lstm_layers"], hyper_parameters["dropout"]).to(processeur)#to(processeur) s'assure que tout s'entraine sur le cuda

#le choix du optimizer et du scheduler a été effectué selon l'article : https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/
"""
On utilise le CTCLoss function dans le speech to text pour aligner les endroits où il est prédit qu'il y ait des lettres avec les bons neurones (Doshi, 2021)
blank permet de ne pas tenir compte des endroits où on prédit qu'il n'y aura pas de caractères : https://distill.pub/2017/ctc/?undefined=&ref=assemblyai.com
"""
criterion = nn.CTCLoss(blank = hyper_parameters["n_caracteres"]).to(processeur) #calcul les probabilités selon une fonction prédéfinie : https://nn.readthedocs.io/en/rtd/criterion/index.html

optimizer = torch.optim.AdamW(rnn.parameters(), hyper_parameters["learning_rate"]) #change les paramètres du modèle pour améliorer la performance : https://towardsdatascience.com/optimizers-for-training-neural-network-59450d71caf6

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hyper_parameters["learning_rate"],
                                                epochs=hyper_parameters["epochs"], # paramètres nécessaires au scheduler :https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.OneCycleLR.html
                                                steps_per_epoch=len(rvd) # nombre de neurones qu'on entraîne par epoch : https://community.deeplearning.ai/t/request-for-explanation-on-steps-per-epoch-parameter/501777
                                                ) #modifie le learning rate pour améliorer la performance : https://towardsdatascience.com/learning-rate-scheduler-d8a55747dd90

texte_transforme = TextTransform(char_map_str)

for epoch in range(hyper_parameters["epochs"]):
  entrainerEpoque(rnn, train_data_loader, criterion, optimizer, scheduler, epoch, processeur)
  if epoch % 2 == 0 :
    tester(rnn, test_data_loader, criterion, epoch, processeur, texte_transforme, hyper_parameters["n_caracteres"])


torch.save(rnn.state_dict(), "feedforwardnet.pth") #sauvegarder le modele : https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html
print("RNN entraîner sauvegardé sur feedforwardnet.pth")

Utiliation du processeur cuda
15 93


RuntimeError: Failed to open the input "M" (No such file or directory).
Exception raised from get_input_format_context at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/stream_reader/stream_reader.cpp:42 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7afbce77fd87 in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x7afbce73075f in /usr/local/lib/python3.10/dist-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42904 (0x7afbce2ca904 in /usr/local/lib/python3.10/dist-packages/torio/lib/libtorio_ffmpeg4.so)
frame #3: torio::io::StreamingMediaDecoder::StreamingMediaDecoder(std::string const&, std::optional<std::string> const&, std::optional<std::map<std::string, std::string, std::less<std::string>, std::allocator<std::pair<std::string const, std::string> > > > const&) + 0x14 (0x7afbce2cd304 in /usr/local/l

# Inférence